<a href="https://colab.research.google.com/github/YBIGTA-VacationProject/DaconComputerVision/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
# !cp "/content/gdrive/MyDrive/data/data_2.zip" "data_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제


!unzip "/content/gdrive/MyDrive/data/data_2.zip"

Archive:  /content/gdrive/MyDrive/data/data_2.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: mnist_data.zip          
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
from typing import Tuple, Sequence, Callable
import glob
from sklearn.model_selection import KFold
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

In [ ]:
labels_df = pd.read_csv('/content/dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('/content/dirty_mnist/*')))[:]
labels = np.array(labels_df.values[:,1:])

In [ ]:
test_imgs_dir = np.array(sorted(glob.glob('/content/test_dirty_mnist/*')))

In [ ]:
labels_df

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,0
49996,49996,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1
49997,49997,0,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0
49998,49998,0,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0


In [ ]:
class MnistDataset(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs_dir)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass

In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Normalize(
    [0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225]
)

In [ ]:
train_dataset = MnistDataset(imgs_dir=imgs_dir, labels=labels, transform=train_transform)

In [ ]:
train_dataset.__getitem__(1)[0]

In [ ]:
resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)

class Resnext(nn.Module):
    def __init__(self):
        super(Resnext, self).__init__()
        self.resnext = resnext 
        self.FC = nn.Linear(1000, 26)
        nn.init.xavier_normal_(self.FC.weight)
      
        

    def forward(self, x):
        x = self.resnext(x)
        x = torch.sigmoid(self.FC(x))
        return x

model = Resnext()
model

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


Resnext(
  (resnext): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seque

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(labels_df):
    folds.append((train_idx, valid_idx))

In [ ]:
train_idx

array([    0,     1,     3, ..., 49996, 49998, 49999])

In [ ]:
valid_idx

array([    2,     9,    10, ..., 49981, 49985, 49997])

In [ ]:
train_dataset = MnistDataset(imgs_dir=imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [ ]:
train_dataset.__len__()

40000

In [ ]:
train_loader.__len__()

625

In [ ]:
for fold in range(1):
    model = Resnext().to(device)

    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])
    valid_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])


    epochs=30
    batch_size=64
    
    # Data Loader
    train_dataset = MnistDataset(imgs_dir=imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

    valid_dataset = MnistDataset(imgs_dir=imgs_dir[valid_idx], labels=labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)  
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    decay_steps = (len(train_dataset)//batch_size)*epochs

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0

    best_models=[]

    for epoch in range(epochs):
      with tqdm(train_loader,total=train_loader.__len__(),unit='batch') as train_bar:
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_bar)):
            train_bar.set_description(f"Train Epoch {epoch}")
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_pred = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

            
            y_pred  = y_pred.cpu().detach().numpy()
            y_pred = y_pred>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_pred == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())
            train_acc = np.mean(batch_accuracy_list)
            
            train_bar.set_postfix(train_loss= loss.item(),train_acc = train_acc)

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []

        with torch.no_grad():
          with tqdm(valid_loader,total=valid_loader.__len__(),unit="batch") as valid_bar:
            for n_valid, (X_valid, y_valid) in enumerate((valid_bar)):
                valid_bar.set_description(f"Valid Epoch {epoch}") 
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                val_acc=np.mean(valid_batch_accuracy)
                valid_bar.set_postfix(valid_loss = valid_loss,valid_acc = val_acc)
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))


        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            best_model=model
            path = "/content/gdrive/MyDrive/Colab Notebooks/models/"
            MODEL = "resnext"
            torch.save(best_model, f'{path}{n}_{MODEL}_{valid_loss:2.4f}_epoch_{epoch}.pth')
            valid_best_accuracy = np.mean(valid_batch_accuracy)

    best_models.append(best_model)        

Train Epoch 0:   0%|          | 0/625 [00:00<?, ?batch/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Valid Epoch 0:   0%|          | 0/313 [00:00<?, ?batch/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: UserWarning: To copy construct from a 

In [ ]:
#저장된 model load
#load_models=[]
#model1 = torch.load('/content/gdrive/MyDrive/Colab Notebooks/models/624_resnext_0.5905_epoch_9.pth')
#load_models.append(model1)

In [ ]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
        ])

In [ ]:
submission = pd.read_csv("sample_submission.csv")

for model in best_models:
    with torch.no_grad():
        model.eval()

        test_dataset = MnistDataset(imgs_dir=test_imgs_dir, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()  
                pred_test = model(X_test).cpu().detach().numpy()
                submission.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 157/157 [28:08<00:00, 10.76s/it]


In [ ]:
submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)
submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0,0,1,0,1,1,1,1,1,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,1,0
1,50001,0,1,0,1,1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,0,1,0,0,1,0
2,50002,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1
3,50003,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,0
4,50004,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0
4996,54996,0,1,1,1,0,0,1,0,1,0,0,1,0,0,1,1,1,0,0,1,0,1,0,1,0,0
4997,54997,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,0,1,1,1,0,1,1
4998,54998,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0


In [ ]:
submission.to_csv('Resnext_prediction.csv', index=False)